## 22) Create a feature view with a query which joins 2 or more cache groups. Apply filter on event time field.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import hashlib
from datetime import datetime

from hops import hdfs

2022-06-01 10:35:30,313 INFO: generated new fontManager


In [2]:
import hsfs

# Create a connection
connection = hsfs.connection()

# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [3]:
df_rides = pd.read_csv(hdfs.project_path() + "Resources/rides.csv", index_col=0)

In [4]:
df_fares = pd.read_csv(hdfs.project_path() + "Resources/fares.csv", index_col=0)

In [5]:
df_rides = df_rides.drop(columns=["is_start"])

In [6]:
df_rides.head(3)

,ride_id,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,taxi_id,driver_id
0,1228,1577904560000,0,-73.867676,40.759323,-73.864080,40.763897,3,2013000194,2013000194
1,527,1577890540000,0,-73.857086,40.772793,-73.807335,40.836117,3,2013000122,2013000122
2,394,1577887880000,0,-73.844320,40.789043,-73.765950,40.888790,3,2013000102,2013000102


In [13]:
df_fares.head(3)

,ride_id,taxi_id,driver_id,pickup_datetime,payment_type,tip,tolls,total_fare
0,1,2013000185,2013000185,1577880020000,CASH,33.0,0.0,118.0
1,2,2013000108,2013000108,1577880040000,CARD,14.0,0.0,48.0
2,3,2013000134,2013000134,1577880060000,CASH,12.0,0.0,41.0


In [9]:
df_rides["pickup_datetime"] = df_rides["pickup_datetime"].astype(int)
df_fares["pickup_datetime"] = df_fares["pickup_datetime"].astype(int)

In [10]:
test22_rides_fg = fs.create_feature_group(name="test22_rides_fg",
                                          version=1,
                                          primary_key=["ride_id"],
                                          event_time=["pickup_datetime"],
                                          description="test22_rides_fg",
                                          online_enabled=True)   
test22_rides_fg.save(df_rides)

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/p/123/jobs/named/test22_rides_fg_1_offline_fg_backfill/executions


In [12]:
test22_fares_fg = fs.create_feature_group(name="test22_fares_fg",
                                          version=1,
                                          primary_key=["ride_id"],
                                          event_time=["pickup_datetime"],
                                          description="test22_fares_fg")   
test22_fares_fg.save(df_fares)

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/p/123/jobs/named/test22_fares_fg_1_offline_fg_backfill/executions


In [25]:
from hsfs.feature import Feature

In [26]:
fg_query = test22_fares_fg.select(['total_fare', 'pickup_datetime'])\
                            .join(test22_rides_fg.select_except(['taxi_id',
                                  'driver_id']), on=['ride_id']).filter(Feature("pickup_datetime") > 1578701500000)

In [27]:
fg_query.show(3)

2022-05-29 23:10:24,374 INFO: USE `testing_featurestore`
2022-05-29 23:10:25,053 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`total_fare` `total_fare`, `fg1`.`pickup_datetime` `pickup_datetime`, `fg1`.`ride_id` `join_pk_ride_id`, `fg1`.`pickup_datetime` `join_evt_pickup_datetime`, `fg0`.`ride_id` `ride_id`, `fg0`.`dropoff_datetime` `dropoff_datetime`, `fg0`.`pickup_longitude` `pickup_longitude`, `fg0`.`pickup_latitude` `pickup_latitude`, `fg0`.`dropoff_longitude` `dropoff_longitude`, `fg0`.`dropoff_latitude` `dropoff_latitude`, `fg0`.`passenger_count` `passenger_count`, RANK() OVER (PARTITION BY `fg1`.`ride_id`, `fg1`.`pickup_datetime` ORDER BY `fg0`.`pickup_datetime` DESC) pit_rank_hopsworks
FROM `testing_featurestore`.`test22_fares_fg_1` `fg1`
INNER JOIN `testing_featurestore`.`test22_rides_fg_1` `fg0` ON `fg1`.`ride_id` = `fg0`.`ride_id` AND `fg1`.`pickup_datetime` >= `fg0`.`pickup_datetime`
WHERE `fg0`.`pickup_datetime` > 1578701500000) NA
WHERE `pit_rank_hopsworks` = 1) (

,total_fare,pickup_datetime,ride_id,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,76.0,1578701520000,41076,0,-73.82090,40.81886,-73.862236,40.766250,3
1,32.0,1578701540000,41077,0,-73.87703,40.74742,-73.935480,40.673030,2
2,14.0,1578701560000,41078,0,-73.89991,40.71830,-73.928530,40.681877,2


In [32]:
fg_query.read().count()

2022-05-29 23:11:54,806 INFO: USE `testing_featurestore`
2022-05-29 23:11:55,612 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`total_fare` `total_fare`, `fg1`.`pickup_datetime` `pickup_datetime`, `fg1`.`ride_id` `join_pk_ride_id`, `fg1`.`pickup_datetime` `join_evt_pickup_datetime`, `fg0`.`ride_id` `ride_id`, `fg0`.`dropoff_datetime` `dropoff_datetime`, `fg0`.`pickup_longitude` `pickup_longitude`, `fg0`.`pickup_latitude` `pickup_latitude`, `fg0`.`dropoff_longitude` `dropoff_longitude`, `fg0`.`dropoff_latitude` `dropoff_latitude`, `fg0`.`passenger_count` `passenger_count`, RANK() OVER (PARTITION BY `fg1`.`ride_id`, `fg1`.`pickup_datetime` ORDER BY `fg0`.`pickup_datetime` DESC) pit_rank_hopsworks
FROM `testing_featurestore`.`test22_fares_fg_1` `fg1`
INNER JOIN `testing_featurestore`.`test22_rides_fg_1` `fg0` ON `fg1`.`ride_id` = `fg0`.`ride_id` AND `fg1`.`pickup_datetime` >= `fg0`.`pickup_datetime`
WHERE `fg0`.`pickup_datetime` > 1578701500000) NA
WHERE `pit_rank_hopsworks` = 1) (

total_fare           3
pickup_datetime      3
ride_id              3
dropoff_datetime     3
pickup_longitude     3
pickup_latitude      3
dropoff_longitude    3
dropoff_latitude     3
passenger_count      3
dtype: int64

In [33]:
fv = fs.create_feature_view(
    name='test22',
    query=fg_query,
    label=["total_fare"]
#     transformation_functions=transformation_functions
)

## 23) Create a feature view with a feature group which has no event time and multiple primary keys and with transformation.

In [7]:
df23 = df_rides.copy()

In [8]:
df23 = df23.iloc[:10000, :5]

In [9]:
df23["second_pk"] = - df23["ride_id"] 

In [10]:
df23

,ride_id,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,second_pk
0,1228,1577904560000,0,-73.867676,40.759323,-1228
1,527,1577890540000,0,-73.857086,40.772793,-527
2,394,1577887880000,0,-73.844320,40.789043,-394
3,1366,1577907320000,0,-73.860565,40.768370,-1366
4,1085,1577901700000,0,-73.831210,40.805737,-1085
...,...,...,...,...,...,...
9995,10940,1578098800000,0,-73.910680,40.704590,-10940
9996,10359,1578087180000,0,-73.858990,40.770374,-10359
9997,10053,1578081060000,0,-73.822105,40.817326,-10053
9998,10187,1578083740000,0,-73.805600,40.838326,-10187


In [82]:
test23_fg = fs.create_feature_group(name="test23_fg",
                                    version=1,
                                    primary_key=["ride_id"],
                                    description="test23_fg",
                                    online_enabled=True)   
test23_fg.save(df23)

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/p/123/jobs/named/test23_fg_1_offline_fg_backfill/executions


---

Now we will try to apply transformation functions to df_fares

In [7]:
df_fares

,ride_id,taxi_id,driver_id,pickup_datetime,payment_type,tip,tolls,total_fare
0,1,2013000185,2013000185,1577880020000,CASH,33.0,0.0,118.0
1,2,2013000108,2013000108,1577880040000,CARD,14.0,0.0,48.0
2,3,2013000134,2013000134,1577880060000,CASH,12.0,0.0,41.0
3,4,2013000062,2013000062,1577880080000,CARD,13.0,0.0,44.0
4,5,2013000087,2013000087,1577880100000,CASH,14.0,0.0,46.0
...,...,...,...,...,...,...,...,...
41073,41074,2013000022,2013000022,1578701480000,CARD,10.0,0.0,34.0
41074,41075,2013000048,2013000048,1578701500000,CASH,11.0,0.0,37.0
41075,41076,2013000176,2013000176,1578701520000,CARD,22.0,0.0,76.0
41076,41077,2013000001,2013000001,1578701540000,CASH,10.0,0.0,32.0


In [10]:
test23_fares_fg = fs.create_feature_group(name="test23_fares_fg",
                                    version=1,
                                    primary_key=["ride_id"],
                                    description="test23_fares_fg",
                                    online_enabled=True)   
test23_fares_fg.save(df_fares.loc[:10000])

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/p/123/jobs/named/test23_fares_fg_1_offline_fg_backfill/executions


In [11]:
# Load transformation functions.
min_max_scaler = fs.get_transformation_function(name="min_max_scaler")

# Map features to transformations.
transformation_functions = {
    "tip": min_max_scaler,
    "total_fare": min_max_scaler
}

In [25]:
fg_query23_fares = test23_fares_fg.select_all()

In [26]:
fg_query23_fares.show(3)

2022-05-30 17:32:52,731 INFO: USE `testing_featurestore`
2022-05-30 17:32:53,577 INFO: SELECT `fg0`.`ride_id` `ride_id`, `fg0`.`taxi_id` `taxi_id`, `fg0`.`driver_id` `driver_id`, `fg0`.`pickup_datetime` `pickup_datetime`, `fg0`.`payment_type` `payment_type`, `fg0`.`tip` `tip`, `fg0`.`tolls` `tolls`, `fg0`.`total_fare` `total_fare`
FROM `testing_featurestore`.`test23_fares_fg_1` `fg0`


,ride_id,taxi_id,driver_id,pickup_datetime,payment_type,tip,tolls,total_fare
0,8518,2013000010,2013000010,1578050360000,CARD,4.0,0.0,10.0
1,7091,2013000186,2013000186,1578021820000,CASH,1.0,0.0,57.0
2,5129,2013000052,2013000052,1577982580000,CASH,29.0,0.0,103.0


In [27]:
fv23_fares = fs.create_feature_view(
    name='test23_fares',
    query=fg_query23_fares,
    label=["payment_type"],
    transformation_functions=transformation_functions
)

In [104]:
# fv23.preview_feature_vector(3)

In [46]:
td_metadata = fv23_fares.create_training_dataset(
    description = 'test23_fares',
    data_format = 'csv',
    splits = {'train': 80, 'validation': 20},
    train_split = "train",
    write_options = {'wait_for_job': False}
)

Training dataset job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/p/123/jobs/named/test23_fares_2_1_create_fv_td_30052022194057/executions


In [47]:
td_version, retrieved_df = fv23_fares.get_training_dataset()

retrieved_df.head()

2022-05-30 19:41:03,621 INFO: USE `testing_featurestore`
2022-05-30 19:41:04,447 INFO: SELECT `fg0`.`ride_id` `ride_id`, `fg0`.`taxi_id` `taxi_id`, `fg0`.`driver_id` `driver_id`, `fg0`.`pickup_datetime` `pickup_datetime`, `fg0`.`payment_type` `payment_type`, `fg0`.`tip` `tip`, `fg0`.`tolls` `tolls`, `fg0`.`total_fare` `total_fare`
FROM `testing_featurestore`.`test23_fares_fg_1` `fg0`


,ride_id,taxi_id,driver_id,pickup_datetime,payment_type,tip,tolls,total_fare
0,8518,2013000010,2013000010,1578050360000,CARD,0.004878,0.0,0.034146
1,7091,2013000186,2013000186,1578021820000,CASH,-0.009756,0.0,0.263415
2,5129,2013000052,2013000052,1577982580000,CASH,0.126829,0.0,0.487805
3,6443,2013000188,2013000188,1578008860000,CASH,0.107317,0.0,0.419512
4,6231,2013000036,2013000036,1578004620000,CASH,-0.014634,0.0,0.121951


In [48]:
td_version

2

In [18]:
retrieved_df.describe()

,ride_id,taxi_id,driver_id,pickup_datetime,tip,tolls,total_fare
count,10001.000000,1.000100e+04,1.000100e+04,1.000100e+04,10001.000000,10001.000000,10001.000000
mean,5001.000000,2.013000e+09,2.013000e+09,1.577980e+12,0.067384,0.261274,0.284120
std,2887.184355,5.773268e+01,5.773268e+01,5.774369e+07,0.050869,0.809833,0.173708
min,1.000000,2.013000e+09,2.013000e+09,1.577880e+12,-0.014634,0.000000,0.000000
25%,2501.000000,2.013000e+09,2.013000e+09,1.577930e+12,0.024390,0.000000,0.151220
50%,5001.000000,2.013000e+09,2.013000e+09,1.577980e+12,0.063415,0.000000,0.268293
75%,7501.000000,2.013000e+09,2.013000e+09,1.578030e+12,0.097561,0.000000,0.395122
max,10001.000000,2.013000e+09,2.013000e+09,1.578080e+12,0.268293,4.000000,1.000000


## 24) Use preview_feature_vector().

In [4]:
df_electricity = pd.read_csv(hdfs.project_path() + "Resources/electricity.csv", index_col=0)

In [10]:
df_electricity = df_electricity.drop(columns=["date"])

In [11]:
df_electricity.head(3)

,RRP,frac_at_neg_RRP,demand,RRP_positive,demand_neg_RRP,RRP_negative,demand_pos_RRP
0,25.633696,0.020833,99635.03,26.415953,2315.790,-7.240000,97319.240
1,33.138988,0.062500,129606.01,38.837661,8523.995,-47.809777,121082.015
2,34.564855,0.000000,142300.54,34.564855,0.000,0.000000,142300.540


In [12]:
df_electricity.shape

(2106, 7)

In [16]:
test24_fg = fs.create_feature_group(name="test24_fg",
                                    version=1,
                                    primary_key=["demand"],
                                    description="test24_fg",
                                    online_enabled=True)   
test24_fg.save(df_electricity)

In [6]:
df_fares.pickup_datetime = df_fares.pickup_datetime.astype(int)

In [7]:
df_fares = df_fares.drop(columns=["payment_type"])

In [8]:
df_fares

,ride_id,taxi_id,driver_id,pickup_datetime,tip,tolls,total_fare
0,1,2013000185,2013000185,1577880020000,33.0,0.0,118.0
1,2,2013000108,2013000108,1577880040000,14.0,0.0,48.0
2,3,2013000134,2013000134,1577880060000,12.0,0.0,41.0
3,4,2013000062,2013000062,1577880080000,13.0,0.0,44.0
4,5,2013000087,2013000087,1577880100000,14.0,0.0,46.0
...,...,...,...,...,...,...,...
41073,41074,2013000022,2013000022,1578701480000,10.0,0.0,34.0
41074,41075,2013000048,2013000048,1578701500000,11.0,0.0,37.0
41075,41076,2013000176,2013000176,1578701520000,22.0,0.0,76.0
41076,41077,2013000001,2013000001,1578701540000,10.0,0.0,32.0


In [10]:
df_fares.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41078 entries, 0 to 41077
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ride_id          41078 non-null  int64  
 1   taxi_id          41078 non-null  int64  
 2   driver_id        41078 non-null  int64  
 3   pickup_datetime  41078 non-null  int64  
 4   tip              41078 non-null  float64
 5   tolls            41078 non-null  float64
 6   total_fare       41078 non-null  float64
dtypes: float64(3), int64(4)
memory usage: 2.5 MB


In [9]:
test24_fares2_fg = fs.create_feature_group(name="test24_fares2_fg",
                                    version=1,
                                    primary_key=["total_fare"],
                                    description="test24_fares2_fg",
#                                        event_time=["pickup_datetime"],
                                    online_enabled=True)   
test24_fares2_fg.save(df_fares)

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/p/123/jobs/named/test24_fares2_fg_1_offline_fg_backfill/executions


FeatureStoreException: The Hopsworks Job failed, use the Hopsworks UI to access the job logs

In [30]:
fv23_fares.preview_feature_vector()

ValueError: Training data version is required for transformation. Call `feature_view.init_serving(version)` to pass the training dataset version.Training data can be created by `feature_view.create_training_data` or `feature_view.get_training_data`.

In [49]:
fv23_fares.init_serving(2)

In [55]:
fv23_fares.preview_feature_vector(5)

[]

In [59]:
# help(fs)

In [50]:
fv23_fares.get_training_data

AttributeError: 'FeatureView' object has no attribute 'get_training_data'

In [32]:
fv23_fares.create_training_data

AttributeError: 'FeatureView' object has no attribute 'create_training_data'

In [28]:
help(fv23_fares.preview_feature_vector)

Help on method preview_feature_vector in module hsfs.feature_view:

preview_feature_vector(external: Union[bool, NoneType] = False) method of hsfs.feature_view.FeatureView instance
    Returns a sample of assembled serving vector from online feature store.
    
    # Arguments
        external: boolean, optional. If set to True, the connection to the
            online feature store is established using the same host as
            for the `host` parameter in the [`hsfs.connection()`](project.md#connection) method.
            If set to False, the online feature store storage connector is used
            which relies on the private IP.
    # Returns
        `list` List of feature values, ordered according to positions of this
        features in training dataset query.



## 25) Create a feature view with a feature group which has a timestamp as event time and with transformation.

In [116]:
fg_query24 = test22_fares_fg.select(['total_fare', 'pickup_datetime'])\
                            .join(test22_rides_fg.select_except(['taxi_id',
                                  'driver_id']), on=['ride_id'])

In [62]:
fg_query24.read()

2022-05-29 23:31:56,862 INFO: USE `testing_featurestore`
2022-05-29 23:31:57,722 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`total_fare` `total_fare`, `fg1`.`pickup_datetime` `pickup_datetime`, `fg1`.`ride_id` `join_pk_ride_id`, `fg1`.`pickup_datetime` `join_evt_pickup_datetime`, `fg0`.`ride_id` `ride_id`, `fg0`.`dropoff_datetime` `dropoff_datetime`, `fg0`.`pickup_longitude` `pickup_longitude`, `fg0`.`pickup_latitude` `pickup_latitude`, `fg0`.`dropoff_longitude` `dropoff_longitude`, `fg0`.`dropoff_latitude` `dropoff_latitude`, `fg0`.`passenger_count` `passenger_count`, RANK() OVER (PARTITION BY `fg1`.`ride_id`, `fg1`.`pickup_datetime` ORDER BY `fg0`.`pickup_datetime` DESC) pit_rank_hopsworks
FROM `testing_featurestore`.`test22_fares_fg_1` `fg1`
INNER JOIN `testing_featurestore`.`test22_rides_fg_1` `fg0` ON `fg1`.`ride_id` = `fg0`.`ride_id` AND `fg1`.`pickup_datetime` >= `fg0`.`pickup_datetime`) NA
WHERE `pit_rank_hopsworks` = 1) (SELECT `right_fg0`.`total_fare` `total_fare`, 

,total_fare,pickup_datetime,ride_id,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,118.0,1577880020000,1,0,-73.76764,40.886640,-73.843834,40.789670,3
1,48.0,1577880040000,2,0,-73.85604,40.774130,-73.802030,40.842870,3
2,41.0,1577880060000,3,0,-73.86453,40.763325,-73.847970,40.784400,3
3,44.0,1577880080000,4,0,-73.86093,40.767902,-73.781784,40.868633,3
4,46.0,1577880100000,5,0,-73.85884,40.770570,-73.754680,40.903137,3
...,...,...,...,...,...,...,...,...,...
41073,34.0,1578701480000,41074,0,-73.87370,40.751648,-73.923590,40.688152,3
41074,37.0,1578701500000,41075,0,-73.92870,40.681652,-73.926890,40.683964,2
41075,76.0,1578701520000,41076,0,-73.82090,40.818860,-73.862236,40.766250,3
41076,32.0,1578701540000,41077,0,-73.87703,40.747420,-73.935480,40.673030,2


In [117]:
# Load transformation functions.
min_max_scaler = fs.get_transformation_function(name="min_max_scaler")

# Map features to transformations.
transformation_functions = {
    "pickup_longitude": min_max_scaler,
    "pickup_latitude": min_max_scaler
}

In [118]:
fv24 = fs.create_feature_view(
    name='test24',
    query=fg_query24,
    label=["total_fare"],
    transformation_functions=transformation_functions
)

In [108]:
td_metadata = fv24.create_training_dataset(
    description = 'test24',
    data_format = 'csv',
    splits = {'train': 80, 'validation': 20},
    train_split = "train",
    write_options = {'wait_for_job': False}
)

Training dataset job started successfully, you can follow the progress at https://hopsworks.glassfish.service.consul:8182/p/123/jobs/named/test24_2_1_create_fv_td_29052022235312/executions


In [68]:
td_version, retrieved_df24 = fv24.get_training_dataset()

retrieved_df24.head()

2022-05-29 23:38:35,404 INFO: USE `testing_featurestore`
2022-05-29 23:38:36,290 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`total_fare` `total_fare`, `fg1`.`pickup_datetime` `pickup_datetime`, `fg1`.`ride_id` `join_pk_ride_id`, `fg1`.`pickup_datetime` `join_evt_pickup_datetime`, `fg0`.`ride_id` `ride_id`, `fg0`.`dropoff_datetime` `dropoff_datetime`, `fg0`.`pickup_longitude` `pickup_longitude`, `fg0`.`pickup_latitude` `pickup_latitude`, `fg0`.`dropoff_longitude` `dropoff_longitude`, `fg0`.`dropoff_latitude` `dropoff_latitude`, `fg0`.`passenger_count` `passenger_count`, RANK() OVER (PARTITION BY `fg1`.`ride_id`, `fg1`.`pickup_datetime` ORDER BY `fg0`.`pickup_datetime` DESC) pit_rank_hopsworks
FROM `testing_featurestore`.`test22_fares_fg_1` `fg1`
INNER JOIN `testing_featurestore`.`test22_rides_fg_1` `fg0` ON `fg1`.`ride_id` = `fg0`.`ride_id` AND `fg1`.`pickup_datetime` >= `fg0`.`pickup_datetime`) NA
WHERE `pit_rank_hopsworks` = 1) (SELECT `right_fg0`.`total_fare` `total_fare`, 

,total_fare,pickup_datetime,ride_id,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,118.0,1577880020000,1,0,-167.092796,0.694774,-73.843834,40.789670,3
1,48.0,1577880040000,2,0,-167.222162,0.530125,-73.802030,40.842870,3
2,41.0,1577880060000,3,0,-167.234587,0.514312,-73.847970,40.784400,3
3,44.0,1577880080000,4,0,-167.229318,0.521010,-73.781784,40.868633,3
4,46.0,1577880100000,5,0,-167.226260,0.524915,-73.754680,40.903137,3


## 27) Delete a feature view.

In [70]:
fv23.delete()

## 28) Get_feature_vector - single id

In [120]:
fg_query27= test22_fares_fg.select_all()

In [ ]:
fv24= fs.create_feature_view(
    name='test24',
    query=fg_query24,
    label=["total_fare"],
    transformation_functions=transformation_functions
)

In [42]:
help(fv23_fares.get_feature_vectors)

Help on method get_feature_vectors in module hsfs.feature_view:

get_feature_vectors(entry: Dict[str, List[Any]], external: Union[bool, NoneType] = False) method of hsfs.feature_view.FeatureView instance
    Returns assembled serving vectors in batches from online feature store.
    
    # Arguments
        entry: dict of feature group primary key names as keys and value as list of primary keys provided by
            serving application.
        external: boolean, optional. If set to True, the connection to the
            online feature store is established using the same host as
            for the `host` parameter in the [`hsfs.connection()`](project.md#connection) method.
            If set to False, the online feature store storage connector is used
            which relies on the private IP.
    # Returns
        `List[list]` List of lists of feature values related to provided primary keys, ordered according to
        positions of this features in training dataset query.



In [45]:
fv23_fares.get_feature_vectors({"ride_id": [1, 2 ,3]})

TypeError: 'NoneType' object is not subscriptable